In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, auc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table_experiments as dt
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px

sns.set_theme(style='darkgrid', palette='dark')

C:\Users\Eric\AppData\Local\Temp\ipykernel_180\2404486966.py:12: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Eric\AppData\Local\Temp\ipykernel_180\2404486966.py:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
model = mlflow.sklearn.load_model('final_model')
df = pd.read_csv('data_after_featuring.csv').drop(columns = ['Unnamed: 0'])
df2 = pd.read_csv('main_info_client.csv').drop(columns = ['Unnamed: 0'])


In [47]:
# Afficher la prédiction d'un client via son ID
# Afficher la description du client via son ID
# Comparer le client avec des clients similaires


# Initialize the app
app = Dash(__name__)

# App layout

app.layout = html.Div([
    html.Div("Enter client id and submit"),
    html.Br(),
    
    dcc.Input(
        type = 'text',
        value = 100013,
        persistence=True,
        id = 'textbox'),
    html.Br(),
    
    html.Div([
        html.Button(id='submit-button',                
                children='Submit'
    )
    ]), 
    html.Br(),
    html.Div(id='output_div_1'),
    html.Br(),

    
    dash_table.DataTable(id='tbl'),
    html.Br(),
    html.Div(id='output_div_2')


])

@app.callback(Output('output_div_1','children'),
              Output('tbl', 'data'),
              Output('output_div_2', 'children'),
              Input('textbox', 'value'),
              Input('submit-button','n_clicks'),
              State('submit-button','n_clicks'))

def update_datatable(id_client, n_clicks,csv_file):            
    if n_clicks:                            
        data = df2.loc[df2['SK_ID_CURR'] == int(id_client)]
        data = data.to_dict('records') 
        prediction = model.predict_proba(df.loc[df['SK_ID_CURR'] ==  int(id_client)].drop(columns = ['SK_ID_CURR', 'TARGET']))

        return (f'infos on client n°{id_client} :', data, f'Prediction for the client as probability of giving the loan : {prediction[0][0]}')



# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jun/2023 03:05:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 03:05:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 03:05:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 03:05:57] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 03:05:57] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
[2023-06-25 03:05:57,762] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Eric\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Eric\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Eric\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv =

In [85]:
# A revoir pour faire sur df2 directement.
from sklearn.preprocessing import StandardScaler 

def find_similar_clients(df2, client_id):
    df = df2.copy()
    df = df.dropna()
    scaler = StandardScaler()
    columns_to_scale = [col for col in df.drop(columns = 'SK_ID_CURR').columns if (df[col].dtype != df.NAME_CONTRACT_TYPE.dtype) ]
    columns_scaled = scaler.fit_transform(df[columns_to_scale])
    df[columns_to_scale] = columns_scaled
    nbrs = NearestNeighbors(n_neighbors=10).fit(columns_scaled)
    indices = nbrs.kneighbors(X = df.loc[scaled_df['SK_ID_CURR'] == client_id][columns_to_scale],
                              n_neighbors = 10,
                              return_distance=False)
    return df2.iloc[indices[0]]

In [86]:
find_similar_clients(df2, 100002)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices